# Parse NOC to ISO data from wikipedia
Because NOC in dataframe differentiate t.ex Soviet Union, EUN, and Russia, we prefer to use ISO to plot on map. For Soviet Union, EUN and Russia, they have the same country: Russia. 

We therefore parse noc to iso dataframe from wikipedia.
However, one question is to get red of the reference number in the country column.

We therefore apply the following method:
[Reference][link]: Pandas read_html() ignore superscripts and subscripts

[link]:https://stackoverflow.com/questions/64309887/pandas-read-html-ignore-superscripts-and-subscripts

In [1]:
import pandas as pd
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://simple.wikipedia.org/wiki/Comparison_of_IOC,_FIFA,_and_ISO_3166_country_codes'

soup = BeautifulSoup(requests.get(url).content, 'html.parser')

# remove <sup>
for sup in soup.select('sup'):
    sup.extract()

table = pd.read_html(str(soup))[0]
table.head()

,Flag,Country,IOC,FIFA,ISO
0,NaN,Afghanistan,AFG,AFG,AFG
1,NaN,Åland Islands,NaN,NaN,ALA
2,NaN,Albania,ALB,ALB,ALB
3,NaN,Algeria,ALG,ALG,DZA
4,NaN,American Samoa,ASA,ASA,ASM


In [2]:
# select relevant columns
table = table.iloc[:, [1, 2, 4]]
table = table.rename(columns={"IOC": "NOC"})
noc_iso = table[table["NOC"].notna()]
noc_iso.head()

,Country,NOC,ISO
0,Afghanistan,AFG,AFG
2,Albania,ALB,ALB
3,Algeria,ALG,DZA
4,American Samoa,ASA,ASM
5,Andorra,AND,AND


- add rows to dataframe
- where both URS(soviet union), EUN([Unified Team at the Olympics][link]) are the regions Russia

[link]: https://en.wikipedia.org/wiki/Unified_Team_at_the_Olympics

In [3]:
data = {'Country': ['Russia', 'Russia', 'Germany'],
	'NOC': ['URS', 'EUN', 'GDR'],
	'ISO': ['RUS', 'RUS', 'DDR']}
new_row = pd.DataFrame(data)
noc_iso = noc_iso.append(new_row, ignore_index=True)
noc_iso.tail()

,Country,NOC,ISO
203,Zambia,ZAM,ZMB
204,Zimbabwe,ZIM,ZWE
205,Russia,URS,RUS
206,Russia,EUN,RUS
207,Germany,GDR,DDR


In [4]:
noc_iso.to_csv("data/noc_iso.csv")

In [5]:
# Import data
athlete_regions = pd.read_csv("data/athlete_regions.csv")
athlete_regions = athlete_regions.iloc[:, :-2]
athlete_regions.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands


In [6]:
# get noc_iso data
noc_iso = pd.read_csv("data/noc_iso.csv")
noc_iso = noc_iso.iloc[:, 1:]
noc_iso.tail()

,Country,NOC,ISO
203,Zambia,ZAM,ZMB
204,Zimbabwe,ZIM,ZWE
205,Russia,URS,RUS
206,Russia,EUN,RUS
207,Germany,GDR,DDR


In [7]:
athlete_iso = athlete_regions.merge(noc_iso, on="NOC", how="left")
athlete_iso = athlete_iso.sort_values(by=["NOC"]).reset_index(drop=True)
athlete_iso[athlete_iso["NOC"]=="URS"]
athlete_iso.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,Country,ISO
0,1076,Jammal-ud-Din Affendi,M,28.0,NaN,NaN,Afghanistan,AFG,1936 Summer,1936,Summer,Berlin,Hockey,Hockey Men's Hockey,NaN,Afghanistan,Afghanistan,AFG
1,133717,Ali Ahmed Youssef Zaid,M,20.0,168.0,58.0,Afghanistan,AFG,1960 Summer,1960,Summer,Roma,Athletics,Athletics Men's 200 metres,NaN,Afghanistan,Afghanistan,AFG
2,502,Ahmad Shah Abouwi,M,NaN,NaN,NaN,Afghanistan,AFG,1956 Summer,1956,Summer,Melbourne,Hockey,Hockey Men's Hockey,NaN,Afghanistan,Afghanistan,AFG
3,5841,Sayed Ali Atta,M,22.0,NaN,NaN,Afghanistan,AFG,1936 Summer,1936,Summer,Berlin,Hockey,Hockey Men's Hockey,NaN,Afghanistan,Afghanistan,AFG
4,1745,Mohammad Aktar,M,17.0,156.0,48.0,Afghanistan,AFG,1980 Summer,1980,Summer,Moskva,Wrestling,"Wrestling Men's Light-Flyweight, Freestyle",NaN,Afghanistan,Afghanistan,AFG


In [8]:
# save athletes_iso
athlete_iso.to_csv("data/athlete_iso.csv")